In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [2]:
# @tool
# def multiply(a: int, b:int) -> int:
#     """ Given 2 numbers a and b this tool returns their product"""
#     return a*b

In [3]:
# print(multiply.invoke({'a': 3, 'b': 4}))

In [4]:
# multiply.name

In [5]:
# multiply.description

In [6]:
# multiply.args

In [7]:
# llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [8]:
# llm.invoke('hi')

In [9]:
# llm_with_tools = llm.bind_tools([multiply])

In [10]:
# llm_with_tools.invoke('Hi how are you')

In [11]:
# query = HumanMessage('Can you multiply 3 with 1000')

In [12]:
# messages = [query]

In [13]:
# messages

In [14]:
# result = llm_with_tools.invoke(messages)

In [15]:
# messages.append(result)

In [16]:
# messages

In [17]:
# tool_result = multiply.invoke(result.tool_calls[0])

In [18]:
# tool_result

In [19]:
# messages.append(tool_result)

In [20]:
# messages

In [21]:
# llm_with_tools.invoke(messages).content

In [46]:
# messages = []

In [4]:
@tool 
def get_conversion_factor(base_currency: str, target_currency: str) -> float: 
    """ 
    The function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/f62a7d2b3fe5e96d1023d4bd/pair/{base_currency}/{target_currency}'
    
    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: float) -> float:
    """
    Given a currency rate this function calculates the target currency value from a given base currency value
    """

    return base_currency_value * conversion_rate

In [5]:
# tool binding
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [49]:
llm_with_tools = llm.bind_tools([
    get_conversion_factor,
    convert
])

In [50]:
messages = [HumanMessage('What is the conversion factor between usd and inr, and based on that can you convert 10 USD to INR?')]

In [51]:
messages

[HumanMessage(content='What is the conversion factor between usd and inr, and based on that can you convert 10 USD to INR?', additional_kwargs={}, response_metadata={})]

In [52]:
ai_message1 = llm_with_tools.invoke(messages)

In [53]:
ai_message1.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '1067c404-a7b0-44f0-b37d-be4bbc33f095',
  'type': 'tool_call'}]

In [55]:
messages.append(ai_message1)

In [57]:
tool_message1 = get_conversion_factor.invoke(ai_message1.tool_calls[0]['args'])

In [59]:
conversion_rate = tool_message1['conversion_rate']
conversion_rate

90.1695

In [61]:
from langchain_core.messages import ToolMessage

messages.append(ToolMessage(
    tool_call_id = ai_message1.tool_calls[0]['id'],
    content=str(conversion_rate),
    name=ai_message1.tool_calls[0]['name']
))

In [63]:
ai_message2 = llm_with_tools.invoke(messages)
ai_message2

AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10, "conversion_rate": 90.1695}'}, '__gemini_function_call_thought_signatures__': {'a5502ba0-735d-496b-a45d-62ca9cf81957': 'CvABAXLI2nzXKtDm26zPhK01wCjS1ZHtGWA+UCsEQVb+bDojMkimwi0DgWucmidZ+HafNNabgLw/10h1H9JlBmO255dnEPKAwdUm1CFcwQfKc9tL05OY2xboFF5w6oNzRJ68gMMuK5Us7uIEJZTxzpoIog+f2sLtrSxBQYQ8g6AhD7rKu2xdrR/9c7LKjjYR5Kge0dfdGCUSfw0waptAvnmWcK2IBAK+ScB3Ga71f8ZL5As9/in2wDFNKg8mRJPLaonwZ3CCid5cYtMtE8KhEaH/ELlDKRv+Z8q6be/EHbgHlauTSHGSlatZ6xzpUziKdOAC'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b03f4-df81-7d23-bc61-7af2d1d07335-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 10, 'conversion_rate': 90.1695}, 'id': 'a5502ba0-735d-496b-a45d-62ca9cf81957', 'type': 'tool_call'}], usage_metadata=

In [64]:
messages.append(ai_message2)

In [65]:
ai_message2.tool_calls

[{'name': 'convert',
  'args': {'base_currency_value': 10, 'conversion_rate': 90.1695},
  'id': 'a5502ba0-735d-496b-a45d-62ca9cf81957',
  'type': 'tool_call'}]

In [ ]:
answer = convert.invoke(ai_message2.tool_calls[0]['args'])
answer

901.6949999999999

In [69]:
messages.append(ToolMessage(
    tool_call_id = ai_message2.tool_calls[0]['id'],
    content= str(answer),
    name= ai_message2.tool_calls[0]['name']
))

In [70]:
response = llm_with_tools.invoke(messages)

In [73]:
response.content[0]['text']

'The conversion factor between USD and INR is 90.1695.\nBased on this, 10 USD is equal to 901.695 INR.'

In [ ]:
from langchain.agents import create_agent

In [7]:
agent = create_agent(
    model = llm, 
    tools=[get_conversion_factor, convert]
)

In [9]:
result = agent.invoke(
    {'messages': [{'role': 'user', 'content': 'What is the conversion factor between usd and inr, and based on that can you convert 10 USD to INR?'}]}
)

In [10]:
result

{'messages': [HumanMessage(content='What is the conversion factor between usd and inr, and based on that can you convert 10 USD to INR?', additional_kwargs={}, response_metadata={}, id='9c1f0dfc-4c0f-4834-b3ff-651977ae923e'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}, '__gemini_function_call_thought_signatures__': {'b4c11e5c-e662-40d5-9564-5dcaab102dd9': 'CuECAXLI2ny9CqedyazPo+m8ifojahbBAXG9g/wDPoCoOOVSYCxdlSwE3JkdlZvEnbIyrr6XqYfvbMk+2bii0sPHiWDNDgZUc5aA4RYwtMGex6KOiHrHbHVNyARlqNpV0TafONjsLfZdL+HGFZSxGuR7R28HPdxT9xbeW2REY0LY7RCVAzm6II30Po7tpwgi+qrkRc/BVEtJru/bKzccOR74lHqy+9AWWjMElyeSLMLCyG2ZC+ZkjJ8iBIa0fx2m3zu+ix8VGD5aX2QtyQKZD7XpvQeFtsRBr6rCzb2Xj1G70lA2WYrEpYacRD+ytVblst6p50zW0RpcmkjtjiapacwmWo/qWBvWjvxgACp6fLYTLstlQFBFqlq+Fwo0UrJfxiZ5Bt53ekeSrIPxgj0p31LcrzEtXsVHowOGkxSYtETetPeo9Ywih/uKdMh7Wt9A6ATBummbGQxL3p+CIuMDUf5KI6M='}}, response_metadata={'prompt_feedback': {'bl

In [11]:
result['messages'][-1].content

'The conversion factor between USD and INR is 90.1695.\n10 USD is equal to 901.695 INR.'